# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
import os
import sys
import datetime

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('flopy version: {}'.format(flopy.__version__))

flopy is installed in c:\users\jlarsen\desktop\flopy-dev\flopy
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
flopy version: 3.3.1


Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("..", "data", "freyberg_multilayer_transient")
ml = flopy.modflow.Modflow.load(nam_file, model_ws=model_ws, check=False)

We can see the ``Modelgrid`` instance has generic entries, as does ``start_datetime``

In [3]:
ml.modelgrid

xll:622241.1904510253; yll:3343617.741737109; rotation:15.0; proj4_str:+proj=utm +zone=14 +ellps=WGS84 +datum=WGS84 +units=m +no_defs; units:meters; lenuni:2

In [4]:
ml.modeltime.start_datetime

'1/1/2015'

Setting the attributes of the ``ml.modelgrid`` is easy:

In [5]:
proj4_str = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
ml.modelgrid.set_coord_info(xoff=123456.7, yoff=765432.1, angrot=15.0, proj4=proj4_str)
ml.dis.start_datetime = '7/4/1776'

In [6]:
ml.modeltime.start_datetime

'7/4/1776'

### Some netCDF export capabilities:

#### Export the whole model (inputs and outputs)

In [7]:
# make directory
pth = os.path.join('data', 'netCDF_export')
if not os.path.exists(pth):
    os.makedirs(pth)

In [8]:
fnc = ml.export(os.path.join(pth, ml.name+'.in.nc'))
hds = flopy.utils.HeadFile(os.path.join(model_ws,"freyberg.hds"))
flopy.export.utils.output_helper(os.path.join(pth, ml.name+'.out.nc'), ml, {"hds":hds})

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)


#### export a single array to netcdf or shapefile

In [9]:
# export a 2d array
ml.dis.top.export(os.path.join(pth, 'top.nc'))
ml.dis.top.export(os.path.join(pth, 'top.shp'))

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
wrote data\netCDF_export\top.shp


#### sparse export of stress period data for a boundary condition package  
* excludes cells that aren't in the package (aren't in `package.stress_period_data`)  
* by default, stress periods with duplicate parameter values (e.g., stage, conductance, etc.) are omitted
(`squeeze=True`); only stress periods with different values are exported  
* argue `squeeze=False` to export all stress periods

In [10]:
ml.drn.stress_period_data.export(os.path.join(pth, 'drn.shp'), sparse=True)

wrote data\netCDF_export\drn.shp


#### Export a 3d array

In [11]:
#export a 3d array
ml.upw.hk.export(os.path.join(pth, 'hk.nc'))
ml.upw.hk.export(os.path.join(pth, 'hk.shp'))

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
wrote data\netCDF_export\hk.shp


#### Export a number of things to the same netCDF file

In [12]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(pth, 'test.nc'))
ml.upw.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)


### Export whole packages to a netCDF file

In [13]:
# export mflist
fnc = ml.wel.export(os.path.join(pth, 'packages.nc'))
ml.upw.export(fnc)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-01-08T20:34:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ..\data\freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSWCAWWLT4175
    create_platform: Windows
    create_directory: C:\Users\jlarsen\Desktop\flopy-dev\examples\Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), floa

### Export the whole model to a netCDF

In [14]:
fnc = ml.export(os.path.join(pth, 'model.nc'))
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-01-08T20:34:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ..\data\freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSWCAWWLT4175
    create_platform: Windows
    create_directory: C:\Users\jlarsen\Desktop\flopy-dev\examples\Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), floa

## Export output to netcdf

FloPy has utilities to export model outputs to a netcdf file. Valid output types for export are MODFLOW binary head files, formatted head files, cell budget files, seawat concentration files, and zonebudget output.

Let's use output from the Freyberg model as an example of these functions

In [15]:
# load binary head and cell budget files
fhead = os.path.join(model_ws, 'freyberg.hds')
fcbc = os.path.join(model_ws, 'freyberg.cbc')

hds = flopy.utils.HeadFile(fhead)
cbc = flopy.utils.CellBudgetFile(fcbc)

export_dict = {"hds": hds,
               "cbc": cbc}

# export head and cell budget outputs to netcdf
fnc = flopy.export.utils.output_helper(os.path.join(pth, "output.nc"), ml, export_dict)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-01-08T20:34:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ..\data\freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSWCAWWLT4175
    create_platform: Windows
    create_directory: C:\Users\jlarsen\Desktop\flopy-dev\examples\Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), floa

### Exporting zonebudget output

zonebudget output can be exported with other modflow outputs, and is placed in a seperate group which allows the user to post-process the zonebudget output before exporting.

Here are two examples on how to export zonebudget output with a binary head and cell budget file

__Example 1__: No postprocessing of the zonebudget output

In [16]:
# load the zonebudget output file
zonbud_ws = os.path.join("..", "data", "zonbud_examples")
fzonbud = os.path.join(zonbud_ws, "freyberg_mlt.2.csv")
zon_arrays = flopy.utils.zonbud.read_zbarray(os.path.join(zonbud_ws, "zonef_mlt.zbr"))

zbout = flopy.utils.ZoneBudgetOutput(fzonbud, ml.dis, zon_arrays)
zbout

ZoneBudgetOutput Class
----------------------

Number of zones: 4
Unique zones: 0, 1, 2, 3
Number of buget records: 3291

In [17]:
export_dict = {'hds': hds,
               'cbc': cbc}

fnc = flopy.export.utils.output_helper(os.path.join(pth, "output_with_zonebudget.nc"),
                                       ml, export_dict)

fnc = zbout.export(fnc, ml)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-01-08T20:34:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ..\data\freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSWCAWWLT4175
    create_platform: Windows
    create_directory: C:\Users\jlarsen\Desktop\flopy-dev\examples\Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), floa

A budget_zones variable has been added to the root group and a new zonebudget group has been added to the netcdf file which hosts all of the budget data

__Example 2__: postprocessing zonebudget output then exporting

In [18]:
# load the zonebudget output and get the budget information
zbout = flopy.utils.ZoneBudgetOutput(fzonbud, ml.dis, zon_arrays)
df = zbout.dataframe
df

,kper,kstp,zone,storage,constant head,other zones,total,zone 0,zone 1,zone 2,zone 3,tslen,totim
0,0,0,1,0.000000,-821.281900,-1570.8210,-2392.103,0.0,0.0000,-1530.4220,-40.3993,1.0,1.0
1,0,0,2,0.000000,-648.804700,630.7300,-18.075,0.0,1530.4220,0.0000,-899.6920,1.0,1.0
2,0,0,3,0.000000,-976.232200,940.0920,-36.140,0.0,40.3993,899.6920,0.0000,1.0,1.0
3,1,0,1,218.568500,-816.347300,-1173.2210,-1770.999,0.0,0.0000,-1134.9370,-38.2835,1.0,2.0
4,1,0,2,191.816342,-643.938700,433.6280,-18.493,0.0,1134.9370,0.0000,-701.3090,1.0,2.0
5,1,0,3,193.469146,-970.052000,739.5920,-36.990,0.0,38.2835,701.3090,0.0000,1.0,2.0
6,2,0,1,573.322530,-797.912500,-745.7910,-970.381,0.0,0.0000,-709.5020,-36.2896,1.0,3.0
7,2,0,2,389.565674,-624.991300,219.8980,-15.528,0.0,709.5020,0.0000,-489.6040,1.0,3.0
8,2,0,3,389.774578,-946.724300,525.8940,-31.057,0.0,36.2896,489.6040,0.0000,1.0,3.0
9,3,0,1,229.221930,-793.510400,-1240.7670,-1805.057,0.0,0.0000,-1205.8200,-34.9476,1.0,4.0


Let's calculate a yearly volumetric budget from the zonebudget data

In [19]:
# get a dataframe of volumetric budget information
vol_df = zbout.volumetric_flux()

# add a year field to the dataframe using datetime
start_date = ml.modeltime.start_datetime
start_date = datetime.datetime.strptime(start_date, "%m/%d/%Y")
nzones = len(zbout.zones) - 1

year = [start_date.year] * nzones
for totim in vol_df.totim.values[:-nzones]:
    t = start_date + datetime.timedelta(days=totim)
    year.append(t.year)

vol_df['year'] = year
    
# calculate yearly volumetric change using pandas
totim_df = vol_df.groupby(['year', 'zone'], as_index=False)['totim'].max()
yearly = vol_df.groupby(['year', 'zone'], as_index=False)[['storage', 'constant head', 'other zones',
                                                           'zone 1', 'zone 2', 'zone 3']].sum()
yearly['totim'] = totim_df['totim']
yearly

,year,zone,storage,constant head,other zones,zone 1,zone 2,zone 3,totim
0,1776,1,81203.267170,-134930.451200,-176631.7910,0.0000,-172310.4031,-4321.3803,181.0
1,1776,2,37268.485533,-102758.917473,62223.7540,172310.4031,0.0000,-110086.6556,181.0
2,1776,3,37296.183058,-158237.438437,114408.0385,4321.3803,110086.6556,0.0000,181.0
3,1777,1,91777.777222,-265545.355300,-407281.7640,0.0000,-398769.3120,-8512.4548,546.0
4,1777,2,42820.852999,-200142.961123,150829.2320,398769.3120,0.0000,-247940.0845,546.0
5,1777,3,42818.153555,-312276.958409,256452.5519,8512.4548,247940.0845,0.0000,546.0
6,1778,1,108226.799462,-255186.105500,-378440.1290,0.0000,-371720.4820,-6719.6672,911.0
7,1778,2,44930.564942,-188836.557151,137315.0100,371720.4820,0.0000,-234405.4742,911.0
8,1778,3,44936.970500,-299267.807443,241125.1428,6719.6672,234405.4742,0.0000,911.0
9,1779,1,5348.709896,-129448.085100,-231291.1680,0.0000,-227734.8190,-3556.3570,1097.0


And finally, export the pandas dataframe to netcdf

In [20]:
# process the new dataframe into a format that is compatible with netcdf exporting
zbncf = zbout.dataframe_to_netcdf_fmt(yearly, flux=False)

# export to netcdf
export_dict = {"hds": hds,
               "cbc": cbc,
               "zbud": zbncf}

fnc = flopy.export.utils.output_helper(os.path.join(pth, "output_with_zonebudget.2.nc"),
                                       ml, export_dict)
fnc.nc

initialize_geometry::proj4_str = +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
initialize_geometry::self.grid_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
initialize_geometry::nc_crs = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)
error getting data for cell_by_cell_flowstorage at time 1.0:list index out of range
error getting data for cell_by_cell_flowstorage at time 1097.0:list index out of range


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3, flopy 3.3.1
    date_created: 2020-01-08T20:34:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -25.0
    geospatial_vertical_max: 4.832500457763672
    geospatial_vertical_resolution: variable
    featureType: Grid
    namefile: freyberg.nam
    model_ws: ..\data\freyberg_multilayer_transient
    exe_name: mf2005.exe
    modflow_version: mfnwt
    create_hostname: IGSWCAWWLT4175
    create_platform: Windows
    create_directory: C:\Users\jlarsen\Desktop\flopy-dev\examples\Notebooks
    solver_head_tolerance: -999
    solver_flux_tolerance: -999
    flopy_sr_xll: 123456.7
    flopy_sr_yll: 765432.1
    flopy_sr_rotation: 15.0
    flopy_sr_proj4_str: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs
    start_datetime: 7/4/1776
    dimensions(sizes): time(1097), layer(3), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), floa